# Cào website [QIPEDC](https://qipedc.moet.gov.vn/dictionary)

In [9]:
# import asyncio
# from requests_html import AsyncHTMLSession
# import bs4

# async def fetch_content(url):
#     asession = AsyncHTMLSession()
#     response = await asession.get(url)
#     await response.html.arender()
#     return response.html.html

# url = 'https://qipedc.moet.gov.vn/dictionary'

# try:
#     # Check if there's an existing event loop
#     loop = asyncio.get_running_loop()
#     # If we're in an interactive environment like Jupyter, use 'await'
#     content = await fetch_content(url)
# except RuntimeError:
#     # If no event loop is running, create a new one
#     content = asyncio.run(fetch_content(url))

# # Now you can parse 'content' with BeautifulSoup or another parser
# soup = bs4.BeautifulSoup(content, 'html.parser')

In [10]:
# print(soup.prettify())

In [11]:
# # Find all button elements having class containing 'btn' 'mx-1' 'btn-sm'
# import re
# buttons = soup.find_all('button', class_=re.compile(r'\bbtn\b.*\bmx-1\b.*\bbtn-sm\b'))
# buttons

# # Find the index of the button with class having 'btn-info' in buttons
# for i, button in enumerate(buttons):
#     if 'btn-info' in button['class']:
#         break
# to_click_button = buttons[i+1]
# to_click_button

In [12]:
# # Get all the paragraphs with class 't-a-center f-s-18 f-f-Lato-Black mb-0'
# list_text = [p.text for p in soup.find_all('p', class_='t-a-center f-s-18 f-f-Lato-Black mb-0')]

In [13]:
# # Get all the src attributes of img tags having class m-w-100
# images = [img['src'] for img in soup.find_all('img', class_='m-w-100')][1:]

In [14]:
# def get_vid_link_from_img_link(img_link):
#     return img_link.replace("thumbs", "videos").replace(".png", ".mp4?autoplay=true")

# videos = [get_vid_link_from_img_link(img) for img in images]
# videos

In [15]:
# # Zip the list_text and videos lists together
# text_video = list(zip(list_text, videos))
# text_video[:5]

In [1]:
# Trang web để tìm phiên bản chromedriver:
# https://googlechromelabs.github.io/chrome-for-testing/

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Thiết lập đường dẫn đến ChromeDriver
chrome_driver_path = '/opt/homebrew/bin/chromedriver'

# Cấu hình các tùy chọn cho Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Chế độ không giao diện
options.add_argument('--disable-gpu')  # Tắt tăng tốc GPU để đảm bảo tương thích
options.add_argument('--no-sandbox')  # Chạy không cần sandbox (chỉ cần cho một số hệ thống nhất định)

# Tạo đối tượng Service với đường dẫn đến ChromeDriver
service = Service(executable_path=chrome_driver_path)

# Khởi tạo WebDriver với Service và Options
driver = webdriver.Chrome(service=service, options=options)

# Điều hướng đến trang web mục tiêu
# Đây là URL của trang mà chúng ta muốn thu thập dữ liệu.
url = 'https://qipedc.moet.gov.vn/dictionary'
driver.get(url)

# Khởi tạo một đối tượng WebDriverWait
# Giúp chờ các thành phần trên trang tải xong trước khi thực hiện các hành động tiếp theo.
wait = WebDriverWait(driver, 10)

# Danh sách để lưu trữ toàn bộ dữ liệu đã thu thập (text + video)
all_text_video = []

try:
    while True:
        # Đợi một khoảng thời gian để đảm bảo trang đã tải đầy đủ nội dung.
        # Thời gian này có thể tùy chỉnh dựa trên tốc độ tải của trang web.
        time.sleep(2)  # Đợi 2 giây trước khi xử lý nội dung

        # Sử dụng BeautifulSoup để phân tích nội dung trang hiện tại
        # Lấy mã HTML của trang web từ WebDriver và xử lý với BeautifulSoup.
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Thu thập tất cả đoạn văn bản (text) dựa trên class CSS cụ thể
        # Class CSS "t-a-center f-s-18 f-f-Lato-Black mb-0" xác định các đoạn văn bản cần lấy.
        list_text = [p.text for p in soup.find_all('p', class_='t-a-center f-s-18 f-f-Lato-Black mb-0')]

        # Thu thập tất cả liên kết ảnh (image src) dựa trên class CSS cụ thể
        # Class "m-w-100" xác định các hình ảnh cần lấy, bỏ qua hình ảnh đầu tiên (logo hoặc ảnh không cần thiết).
        images = [img['src'] for img in soup.find_all('img', class_='m-w-100')][1:]

        # Hàm chuyển đổi liên kết ảnh sang liên kết video
        # Do cấu trúc URL video được xây dựng dựa trên liên kết ảnh, chỉ cần thay đổi "thumbs" thành "videos"
        # và thay đổi phần mở rộng từ .png sang .mp4.
        def get_vid_link_from_img_link(img_link):
            return img_link.replace("thumbs", "videos").replace(".png", ".mp4?autoplay=true")

        # Tạo danh sách các liên kết video từ danh sách ảnh
        videos = [get_vid_link_from_img_link(img) for img in images]

        # Ghép các đoạn văn bản và liên kết video thành các cặp (tuple)
        # Sử dụng zip để kết hợp danh sách text và videos.
        text_video = list(zip(list_text, videos))
        
        # Thêm dữ liệu từ trang hiện tại vào danh sách tổng
        all_text_video.extend(text_video)

        # Tìm nút to_click_button để chuyển sang trang tiếp theo
        try:
            # Tìm tất cả các nút có class chứa 'btn' 'mx-1' 'btn-sm'
            buttons = driver.find_elements(By.CSS_SELECTOR, 'button[class*="btn"][class*="mx-1"][class*="btn-sm"]')
            # Tìm chỉ số của nút có class chứa 'btn-info'
            for i, button in enumerate(buttons):
                if 'btn-info' in button.get_attribute('class'):
                    break
            # Lấy nút tiếp theo để chuyển trang
            to_click_button = buttons[i + 1]
            # Nhấp vào nút để chuyển trang
            to_click_button.click()
            # Chờ cho trang tiếp theo tải xong trước khi tiếp tục
            wait.until(EC.staleness_of(to_click_button))
        except:
            # Nếu không tìm thấy nút hoặc gặp lỗi, thoát khỏi vòng lặp
            print("Không còn trang nào để tiếp tục.")
            break

finally:
    # Đảm bảo đóng trình duyệt ngay cả khi xảy ra lỗi
    driver.quit()

# In ra 5 cặp text-video đầu tiên để kiểm tra kết quả
print(all_text_video[:5])

# Nếu mà macOS không cho mở chromedriver thì
# xem cái này https://discussions.apple.com/thread/250425993?sortBy=rank

Không còn trang nào để tiếp tục.
[('địa chỉ', '/videos/D0001B.mp4?autoplay=true'), ('địa chỉ', '/videos/D0001N.mp4?autoplay=true'), ('địa chỉ', '/videos/D0001T.mp4?autoplay=true'), ('tỉnh', '/videos/D0002.mp4?autoplay=true'), ('tiếp tân', '/videos/D0003.mp4?autoplay=true')]


In [2]:
# Export dữ liệu thu thập được ra file CSV
import csv

# Đường dẫn đến file CSV để lưu trữ dữ liệu
csv_file_path = '../data/raw/text_video_data.csv'

# Mở file CSV để ghi dữ liệu
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Ghi tiêu đề cho file CSV
    writer.writerow(['Text', 'Video'])
    # Ghi dữ liệu từ danh sách all_text_video vào file CSV
    writer.writerows(all_text_video)

In [3]:
# Đọc dữ liệu từ file CSV để kiểm tra
import pandas as pd

# Đọc dữ liệu từ file CSV
csv_file_path = '../data/raw/text_video_data.csv'
df = pd.read_csv(csv_file_path)

# In ra 5 dòng đầu tiên của DataFrame
print(df.head())

# Đổi tên cột Video thành Video URL và nối thêm phần url vào mỗi giá trị trong cột Video URL
df.rename(columns={'Video': 'Video URL'}, inplace=True)
df['Video URL'] = 'https://qipedc.moet.gov.vn' + df['Video URL']

# In ra 5 dòng đầu tiên của DataFrame sau khi thay đổi
print(df.head())

       Text                             Video
0   địa chỉ  /videos/D0001B.mp4?autoplay=true
1   địa chỉ  /videos/D0001N.mp4?autoplay=true
2   địa chỉ  /videos/D0001T.mp4?autoplay=true
3      tỉnh   /videos/D0002.mp4?autoplay=true
4  tiếp tân   /videos/D0003.mp4?autoplay=true
       Text                                          Video URL
0   địa chỉ  https://qipedc.moet.gov.vn/videos/D0001B.mp4?a...
1   địa chỉ  https://qipedc.moet.gov.vn/videos/D0001N.mp4?a...
2   địa chỉ  https://qipedc.moet.gov.vn/videos/D0001T.mp4?a...
3      tỉnh  https://qipedc.moet.gov.vn/videos/D0002.mp4?au...
4  tiếp tân  https://qipedc.moet.gov.vn/videos/D0003.mp4?au...


In [4]:
def region_from_video_url(video_url):
    """
    Hàm trích xuất khu vực từ URL video.
    Ví dụ: Video URL: https://qipedc.moet.gov.vn/videos/D0001T.mp4?a có chữ T trước
    .mp4 thì khu vực là 'T'.
    Video URL: https://qipedc.moet.gov.vn/videos/D0002.mp4?au không có chữ gì trước
    .mp4 thì khu vực là None, nghĩa là không xác định.
    Có 3 khu vực có thể có là 'B', 'T', 'N'.
    """
    
    # Tách câu bởi dấu "." cuối cùng
    parts = video_url.split('.')
    
    # Lấy phần kế cuối
    last_part = parts[-2]
    
    # Nếu kí tự cuối của phần kế cuối thuộc ['B', 'T', 'N'] thì trả về kí tự đó
    if last_part[-1] in ['B', 'T', 'N']:
        return last_part[-1]
    else:
        return None

# Thêm cột Region vào DataFrame từ cột Video URL
df['Region'] = df['Video URL'].apply(region_from_video_url)

# In ra 5 dòng đầu tiên của DataFrame sau khi thêm cột Region
print(df.head())

       Text                                          Video URL Region
0   địa chỉ  https://qipedc.moet.gov.vn/videos/D0001B.mp4?a...      B
1   địa chỉ  https://qipedc.moet.gov.vn/videos/D0001N.mp4?a...      N
2   địa chỉ  https://qipedc.moet.gov.vn/videos/D0001T.mp4?a...      T
3      tỉnh  https://qipedc.moet.gov.vn/videos/D0002.mp4?au...   None
4  tiếp tân  https://qipedc.moet.gov.vn/videos/D0003.mp4?au...   None


In [5]:
# Tạo thêm một cột Label là cột Text thay khoảng trắng bằng _ rồi + với cột Region
df['Label'] = df['Text'].str.replace(' ', '_') + df['Region'].fillna('')


# In ra 5 dòng đầu tiên của DataFrame sau khi thêm cột Label
print(df.head())

       Text                                          Video URL Region  \
0   địa chỉ  https://qipedc.moet.gov.vn/videos/D0001B.mp4?a...      B   
1   địa chỉ  https://qipedc.moet.gov.vn/videos/D0001N.mp4?a...      N   
2   địa chỉ  https://qipedc.moet.gov.vn/videos/D0001T.mp4?a...      T   
3      tỉnh  https://qipedc.moet.gov.vn/videos/D0002.mp4?au...   None   
4  tiếp tân  https://qipedc.moet.gov.vn/videos/D0003.mp4?au...   None   

      Label  
0  địa_chỉB  
1  địa_chỉN  
2  địa_chỉT  
3      tỉnh  
4  tiếp_tân  


In [6]:
# Sort lại DataFrame theo cột Text
df.sort_values('Text', inplace=True)

In [7]:
# Lưu dataframe vào lại file .csv cũ
df.to_csv(csv_file_path, index=False)